In [10]:
import os
import pandas as pd
import numpy as np
import librosa
from os import listdir
from os.path import isfile, join, normpath
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
RANDOM_STATE = 26
TEST_SIZE = 0.3
N_MFCC = 60
LS_CSV_PATH = "../data/HLS-CMDS/LS.csv"
LS_AUDIO_PATH = "../data/HLS-CMDS/LS/LS"

In [3]:
df = pd.read_csv(LS_CSV_PATH)
print(df.head())

  Gender Lung Sound Type Location Lung Sound ID
0      M          Normal      RUA       M_N_RUA
1      F          Normal      LUA       F_N_LUA
2      F          Normal      RMA       F_N_RMA
3      F          Normal      LMA       F_N_LMA
4      M          Normal      RLA       M_N_RLA


In [12]:
df.columns

Index(['Gender', 'Lung Sound Type', 'Location', 'Lung Sound ID', 'audio_path'], dtype='object')

In [4]:
df['audio_path'] = df['Lung Sound ID'].apply(lambda x: normpath(join(LS_AUDIO_PATH, f"{str(x).strip()}.wav").replace('\\', '/')))

In [5]:
df['audio_path'] = df['audio_path'].str.replace('_C_', '_FC_').str.replace('_G_', '_CC_')

In [6]:
def extract_mfcc(filename, n_mfcc=N_MFCC):
    y, sr = librosa.load(filename, sr=44100)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_mean = np.mean(mfcc, axis=1)
    mfcc_std = np.std(mfcc, axis=1)
    return np.concatenate([mfcc_mean, mfcc_std])

def flatten_mfcc(mfcc):
    return np.array(mfcc).flatten()

In [7]:
mfcc_list = []
labels = []

for _, row in df.iterrows():
    try:
        mfcc = extract_mfcc(row['audio_path'])
        # mfcc_flat = flatten_mfcc(mfcc)
        mfcc_list.append(mfcc)
        labels.append(row['Lung Sound Type'])
    except Exception as e:
        print(f"[WARN] Failed: {row['audio_path']} -> {e}")
    
X = pd.DataFrame(mfcc_list)
y = pd.Series(labels)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

In [8]:
pipe_model = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('svc', SVC(kernel='rbf', random_state=RANDOM_STATE, class_weight='balanced'))
])

param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf', 'linear', 'poly']
}

model_grid = GridSearchCV(
    estimator=pipe_model, 
    param_grid=param_grid, 
    scoring='f1_macro', 
    cv=5, 
    n_jobs=-1, 
    verbose=2
)

In [9]:
model_grid.fit(Xtrain, ytrain)
model = model_grid.best_estimator_
print(f"Best parameters: {model_grid.best_params_}")

ypred = model.predict(Xtest)
f1 = f1_score(ytest, ypred, average='macro')
print(f"LS: Lung Sound Type -> macro-F1: {f1}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits


c:\Users\ardik\Anaconda3\envs\ExamMachineLearning\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


Best parameters: {'svc__C': 0.1, 'svc__gamma': 0.001, 'svc__kernel': 'linear'}
LS: Lung Sound Type -> macro-F1: 1.0


In [11]:
cm = confusion_matrix(ytest, ypred)
print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(classification_report(ytest, ypred))

Confusion Matrix:
[[3 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 4 0 0 0]
 [0 0 0 3 0 0]
 [0 0 0 0 2 0]
 [0 0 0 0 0 2]]

Classification Report:
                 precision    recall  f1-score   support

Coarse Crackles       1.00      1.00      1.00         3
  Fine Crackles       1.00      1.00      1.00         1
         Normal       1.00      1.00      1.00         4
    Pleural Rub       1.00      1.00      1.00         3
        Rhonchi       1.00      1.00      1.00         2
       Wheezing       1.00      1.00      1.00         2

       accuracy                           1.00        15
      macro avg       1.00      1.00      1.00        15
   weighted avg       1.00      1.00      1.00        15

